In [51]:
# Import Dependencies
import requests
import json
from bs4 import BeautifulSoup
import time
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from pandas import to_datetime

In [104]:
# Read CSV File
filt_data_df = pd.read_csv("tables/main_table1.csv")
filt_data_df

,Unnamed: 0,title,rating,description,language,isbn,genres,pages,publishDate,publisher,numRatings,ratingsByStars,likedPercent,main author
0,0,The Hunger Games,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374,09/14/08,Scholastic Press,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,Suzanne Collins
1,1,Harry Potter and the Order of the Phoenix,4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",870,09/28/04,Scholastic Inc.,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,J.K. Rowling
2,2,Twilight,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...",501,09/06/06,"Little, Brown and Company",4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,Stephenie Meyer
3,3,The Book Thief,4.37,Librarian's note: An alternate cover edition c...,English,9780375831003,"['Historical Fiction', 'Fiction', 'Young Adult...",552,03/14/06,Alfred A. Knopf,1834276,"['1048230', '524674', '186297', '48864', '26211']",96.0,Markus Zusak
4,4,Animal Farm,3.95,Librarian's note: There is an Alternate Cover ...,English,9780451526342,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",141,04/28/96,Signet Classics,2740713,"['986764', '958699', '545475', '165093', '84682']",91.0,George Orwell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35682,35682,Unbelievable,4.16,Lilah Lopez Dreser's in town to take care of u...,English,9781477594247,"['Romance', 'Young Adult', 'Contemporary', 'Co...",360,April 11th 2013,Wordpaintings Unlimited,1028,"['442', '384', '142', '48', '12']",94.0,Sherry Gammon
35683,35683,Fractured,4.00,The Fateful Trilogy continues with Fractured. ...,English,2940012616562,"['Vampires', 'Paranormal', 'Young Adult', 'Rom...",0,May 28th 2011,Cheri Schmidt,871,"['311', '310', '197', '42', '11']",94.0,Cheri Schmidt
35684,35684,Marked,3.70,--READERS FAVORITE AWARDS WINNER 2011--Sixteen...,English,9781461017097,"['Fantasy', 'Young Adult', 'Paranormal', 'Ange...",280,March 18th 2011,CreateSpace,6674,"['2109', '1868', '1660', '647', '390']",84.0,Kim Richardson
35685,35685,Wayward Son,3.85,A POWERFUL TREMOR UNEARTHS AN ANCIENT SECRETBu...,English,9781450755634,"['Fiction', 'Mystery', 'Historical Fiction', '...",507,September 1st 2011,Cascada Productions,238,"['77', '78', '59', '19', '5']",90.0,Tom Pollack


In [ ]:
filt_data_df['publishDate'].value_counts

In [ ]:
# Cleaned the published date column to bring all dates to one format
filt_data_df["publishedDate"]= pd.to_datetime(filt_data_df['publishDate'],errors = 'coerce')
filt_data_df

In [ ]:
filt_data_df=filt_data_df.dropna()

In [ ]:
filt_data_df['yearPublished'] = pd.DatetimeIndex(filt_data_df['publishedDate']).year
filt_data_df

In [ ]:
# Dropped irregular date format column after cleaning
filt_data_df.drop(columns="publishDate")

In [ ]:
# Saving dataframe as csv
filt_data_df.to_csv("tables/year.csv", index= False)

In [ ]:
# Filtering the data for latest 5 yrs
filt_data_df.sort_values(by="yearPublished", ascending=False)

In [ ]:
#df = df[(df['closing_price'] >= 99) & (df['closing_price'] <= 101)]
filt_data_df=filt_data_df[(filt_data_df['yearPublished']>=2011)&(filt_data_df['yearPublished']<=2020)].reset_index(inplace=False)

In [47]:
filt_data_df.count()

title             11868
rating            11868
description       11868
language          11868
isbn              11868
genres            11868
pages             11868
publishDate       11868
publisher         11868
numRatings        11868
ratingsByStars    11868
likedPercent      11868
main author       11868
publishedDate     11868
yearPublished     11868
dtype: int64

In [ ]:
filt_data_df = filt_data_df.drop(columns=["index","Unnamed: 0"])

In [46]:
# Saving dataframe as csv
filt_data_df.to_csv("tables/main_table.csv", index= False)

In [92]:
# Make list of authors
author_list = filt_data_df['main author'].tolist()
author_list1 = author_list[:3084]
author_list2 = author_list[3084:5484]
author_list3 = author_list[5485:7245]
author_list4 = author_list[7564:10633]
author_list5 = author_list[10633:11573]
author_list6 = author_list[7245:7564]
author_list7 = author_list[11573:]


len(author_list7)

295

In [93]:
# Base url
base_url = "https://en.wikipedia.org/wiki/"
author_dict = {}
for author_name in author_list7:
    # create url along with author name
    url = base_url + author_name
    # Retrieve page with the requests module
    response = requests.get(url)
    try:
    # Create BeautifulSoup object; parse with 'html.parser'
        soup = BeautifulSoup(response.text, 'html.parser')
        author_dict[author_name]= soup.find('span', class_="bday").text
    except AttributeError:
        author_dict[author_name]= "none"

   
   
author_dict  

{'Laurann Dohner': 'none',
 'Colum McCann ': '1965-02-28',
 'Ed Brubaker ': '1966-11-17',
 'Jane Lythell ': 'none',
 'Chuck Dixon ': '1954-04-14',
 'Lucy Monroe ': '1906-10-23',
 'Toni Aleo ': 'none',
 'Sam Angus ': 'none',
 'Nancy Guthrie ': '1952-06-15',
 'Jacob Grimm': '1785-01-04',
 'Ruth Axtell ': 'none',
 'Allan Frewin Jones': '1954-04-30',
 'S.T. Boston ': 'none',
 'Gemma James ': 'none',
 'Kristen Iversen ': 'none',
 'Adele Parks ': 'none',
 'Rowan Coleman ': 'none',
 'Pepper Winters ': 'none',
 'Neal Asher ': '1961-02-04',
 'Judith Rossell ': 'none',
 'Lynda Cohen Loigman ': 'none',
 'George R.R. Martin': '1948-09-20',
 'Hope Larson ': 'none',
 'Cora Brent ': 'none',
 'Jon Krakauer ': '1954-04-12',
 'Andrea Bobotis ': 'none',
 'Liz Curtis Higgs ': 'none',
 'J.D. Hughes ': 'none',
 'Anita Stansfield': 'none',
 'Meg Howrey': 'none',
 'Madeleine Wickham': '1970-12-12',
 'Anna Funder': 'none',
 'Jess Epps ': 'none',
 'Bella Forrest ': 'none',
 'Eve Hathaway': 'none',
 'Ava Miles '

In [94]:
len(author_dict)

240

In [95]:
# Converting dictionary to dataframe
df = pd.DataFrame.from_dict(author_dict, orient ='index')
df

,0
Laurann Dohner,none
Colum McCann,1965-02-28
Ed Brubaker,1966-11-17
Jane Lythell,none
Chuck Dixon,1954-04-14
...,...
Kim Richardson,1965-12-22
Sherry Gammon,none
Cheri Schmidt,none
Tom Pollack,none


In [96]:
df.to_csv('author_data/data7.csv', index=True)

In [101]:
df = pd.concat(
    map(pd.read_csv, ['author_data/data1.csv', 'author_data/data2.csv',"author_data/data3.csv","author_data/data4.csv","author_data/data5.csv","author_data/data6.csv","author_data/data7.csv" ]), ignore_index=True)
df = df.rename(columns={'Unnamed: 0': 'Author', '0': 'DOB'})

In [103]:
df.loc[df["DOB"]!="none"]

,Author,DOB
0,J.R.R. Tolkien,1892-01-03
1,Ray Bradbury,1920-08-22
2,Veronica Roth,1988-08-19
3,Paulo Coelho,1947-08-24
4,Audrey Niffenegger,1963-06-13
...,...,...
11795,Marissa Meyer,1984-02-19
11796,Elizabeth Goudge,1900-04-24
11801,Daniel Tammet,1979-01-31
11803,Nagaru Tanigawa,1970-12-19
